<a href="https://colab.research.google.com/github/EsraaAdinar/python-and-cyber/blob/main/aws_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-Use Google's API (a set of tools provided by Google) to find a list of schools near the location provided by the user.

In [ ]:
import os
from google.colab import output
#store the API in envvironment variable and make sure to hide it in the final draft
key=input("enter key")
os.environ['API_KEY'] = key
output.clear()

In [2]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 950.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.4 MB/s eta 0:00:00


# 2-Import the googlemaps library and get a client for accessing the API functions.

In [3]:
!pip install googlemaps
import googlemaps

gmaps = googlemaps.Client(os.environ.get('API_KEY'))

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=d2cebdef7579ea178a4ef315fd267c21fb94ded96c8c163dcfdc492dc93c5a12
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


# 3- Get a list of schools within a radius that is given by the user

In [28]:

def get_user_location(latitude=None, longitude=None, radius=None):
    try:
        #If  latitude, longitude, and radius are not provided, get them from the user
        if latitude is None:
         latitude = float(input("Enter your latitude: "))
        if longitude is None:
         longitude = float(input("Enter your longitude: "))
        if radius is None:
           radius = float(input("Enter the radius (in kilometers): "))
           return latitude, longitude, radius
        #else:
        #  return None

    except ValueError as e:
       print(f"Invalid input. Please enter valid numerical values. Error: {e}")



# Getting nearby schools from giving locations

In [29]:
import json
import boto3
import googlemaps

def get_nearby_schools( latitude, longitude, radius):
    try:
        # Get user location (if not provided as function parameters)
        user_location = get_user_location()

        if user_location:
            latitude, longitude, radius = user_location

            # Initialize the Google Maps API client
            gmaps = googlemaps.Client(key)

            # Perform a Places API text search for schools within the specified radius
            places_result = gmaps.places_nearby(
                location=(latitude, longitude),
                radius=radius * 1000,  # Convert radius to meters
                keyword='school',
                type='school',
            )

            # Extract more information for each school
            schools_list = []
            for place in places_result.get('results', []):
                school_info = {
                    'Name': place.get('name', ''),
                    'Latitude': place['geometry']['location']['lat'],
                    'Longitude': place['geometry']['location']['lng'],
                    'Radius': radius,
                }
                schools_list.append(school_info)

            # Print the schools in dictionary form
            print("List of Schools Near User:")
            for school_info in schools_list:
               print(school_info)
            return schools_list
    except Exception as e:
        print(f"Error: {str(e)}")


schools_list = get_nearby_schools(latitude=None, longitude=None, radius=None)


print(schools_list)

Enter your latitude: 55.816555
Enter your longitude: -4.309890
Enter the radius (in kilometers): 4
List of Schools Near User:
{'Name': 'Shawlands Primary School', 'Latitude': 55.82952239999999, 'Longitude': -4.2855491, 'Radius': 4.0}
{'Name': 'Pollokshields Primary School', 'Latitude': 55.84229389999999, 'Longitude': -4.2747136, 'Radius': 4.0}
{'Name': 'Hillington Primary School', 'Latitude': 55.8524158, 'Longitude': -4.3582729, 'Radius': 4.0}
{'Name': 'Sandwood Primary School', 'Latitude': 55.8495065, 'Longitude': -4.3621152, 'Radius': 4.0}
{'Name': 'Sir John Maxwell School', 'Latitude': 55.8255717, 'Longitude': -4.2970815, 'Radius': 4.0}
{'Name': 'Cuthbertson Primary School', 'Latitude': 55.8397751, 'Longitude': -4.2651444, 'Radius': 4.0}
{'Name': 'Ibrox Primary School & Nursery Class', 'Latitude': 55.8517114, 'Longitude': -4.3084599, 'Radius': 4.0}
{'Name': 'Ashpark Primary School', 'Latitude': 55.8077353, 'Longitude': -4.3329298, 'Radius': 4.0}
{'Name': 'Tinto Primary School', 'Lat

# set environment virables for keys and bucket

In [6]:
import os
from IPython.display import clear_output

def set_environment_variable_values():
  ACCESS_KEY = input("Please enter the AWS access key: ")
  SECRET_ACCESS_KEY = input("Please enter the AWS secret access key: ")
  BUCKET_NAME = input("Please enter the name of the bucket in S3: ")
  os.environ['ACCESS_KEY'] = ACCESS_KEY
  os.environ['SECRET_ACCESS_KEY'] = SECRET_ACCESS_KEY
  os.environ['BUCKET_NAME'] = BUCKET_NAME
  clear_output()
  return None

set_environment_variable_values()


# set client connection

In [30]:
import os
import json
import boto3

def get_S3_client():
    resource = boto3.client(
        "s3",
        aws_access_key_id=os.environ.get('ACCESS_KEY'),
        aws_secret_access_key=os.environ.get('SECRET_ACCESS_KEY')
    )
    return resource


def save_schools_to_s3(schools_list, s3_bucket_name, s3_key):
    try:
        # Get S3 client
        s3_client = get_S3_client()

        if schools_list:
            # Define CSV file header
            csv_header = schools_list[0].keys()

            # Convert list of dictionaries to a CSV-formatted string
            csv_data = ','.join(csv_header) + '\n'
            csv_data += '\n'.join(','.join(str(item[key]) for key in csv_header) for item in schools_list)

            # Upload CSV file to S3
            s3_client.put_object(
                Bucket=s3_bucket_name,
                Key=s3_key,
                Body=csv_data,
                ContentType='text/csv'
            )

            print("List of schools saved to S3 at s3.")
        else:
            print("Error: The list of schools is empty.")
    except Exception as e:
        print(f"Error: {str(e)}")


schools_list = get_nearby_schools(latitude=None, longitude=None, radius=None)
s3_bucket_name = 'aws-project-esraa'
s3_key = 'schools_list.csv'

save_schools_to_s3(schools_list, s3_bucket_name, s3_key)


Enter your latitude: 55.816555
Enter your longitude: -4.309890
Enter the radius (in kilometers): 4
List of Schools Near User:
{'Name': 'Shawlands Primary School', 'Latitude': 55.82952239999999, 'Longitude': -4.2855491, 'Radius': 4.0}
{'Name': 'Pollokshields Primary School', 'Latitude': 55.84229389999999, 'Longitude': -4.2747136, 'Radius': 4.0}
{'Name': 'Hillington Primary School', 'Latitude': 55.8524158, 'Longitude': -4.3582729, 'Radius': 4.0}
{'Name': 'Sandwood Primary School', 'Latitude': 55.8495065, 'Longitude': -4.3621152, 'Radius': 4.0}
{'Name': 'Sir John Maxwell School', 'Latitude': 55.8255717, 'Longitude': -4.2970815, 'Radius': 4.0}
{'Name': 'Cuthbertson Primary School', 'Latitude': 55.8397751, 'Longitude': -4.2651444, 'Radius': 4.0}
{'Name': 'Ibrox Primary School & Nursery Class', 'Latitude': 55.8517114, 'Longitude': -4.3084599, 'Radius': 4.0}
{'Name': 'Ashpark Primary School', 'Latitude': 55.8077353, 'Longitude': -4.3329298, 'Radius': 4.0}
{'Name': 'Tinto Primary School', 'Lat

In [31]:
def get_file(filename):
    try:
        # get the file from the bucket
        s3_client = get_S3_client()
        file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)

        # read the data from the file object and decode it from bytes data to string data
        data_file = file_object['Body'].read()
        return str(data_file.decode("utf-8"))
    except Exception as e:
        print(f"Error fetching file '{filename}': {str(e)}")
        return None


data = get_file('schools_list.csv')
if data:
    print(data)
else:
    print("Failed to retrieve data from file.")


Name,Latitude,Longitude,Radius
Shawlands Primary School,55.82952239999999,-4.2855491,4.0
Pollokshields Primary School,55.84229389999999,-4.2747136,4.0
Hillington Primary School,55.8524158,-4.3582729,4.0
Sandwood Primary School,55.8495065,-4.3621152,4.0
Sir John Maxwell School,55.8255717,-4.2970815,4.0
Cuthbertson Primary School,55.8397751,-4.2651444,4.0
Ibrox Primary School & Nursery Class,55.8517114,-4.3084599,4.0
Ashpark Primary School,55.8077353,-4.3329298,4.0
Tinto Primary School,55.8175865,-4.292825,4.0
Cardonald Primary School,55.8422919,-4.3503377,4.0
Mosspark Primary School,55.8391122,-4.3276337,4.0
Braidbar Primary School,55.8065805,-4.2837185,4.0
Merrylee Primary School,55.8166313,-4.274818199999999,4.0
Holy Cross Primary School,55.836191,-4.254701499999999,4.0
Giffnock Primary School,55.80277239999999,-4.2937503,4.0
Darnley Primary School,55.8079233,-4.33972,4.0
Hollybrook Academy,55.8368914,-4.253457,4.0
Gowanbank Primary School,55.8172089,-4.363136,4.0
Langside Primary Sch

# 5-Lamda Handler function

In [43]:
import os
import json
#from functions import get_S3_client, get_file

def lambda_handler(event, context):
    try:
        request_body = event.get('body', {})
        action = request_body.get('data', {}).get('action')
        latitude = request_body.get('data', {}).get('latitude')
        longitude = request_body.get('data', {}).get('longitude')
        radius = request_body.get('data', {}).get('radius')
        datalist = request_body.get('data', {}).get('datalist')
        csv_filename= request_body.get('data',{}).get('csv_filename')
        if "body" in event.keys():
            request = event["body"]
            if type(request) is not dict:
                request = json.loads(request)

            if request is not None and "data" in request.keys():
                data = request["data"]
                operation = data['action']
                if operation == "store":
                    schools_list = get_nearby_schools(latitude, longitude, radius)
                    s3_bucket_name = os.environ.get('BUCKET_NAME')
                    s3_key = f'{datalist}.csv'
                    save_schools_to_s3(schools_list, s3_bucket_name, s3_key)

                    return {
                        "statusCode": 200,
                        "body": json.dumps({"message": "School locations retrieved and saved to S3"})
                    }
                elif operation == "get":
                  # csv_filename = data["csv_filename"]
                   return_data = get_file(csv_filename)
                   return {
                        "statusCode": 200,
                        "headers":{
                        "Content-Type" : "application/json",
                        "Access-Control-Allow-Headers" : "Content-Type,X-Api-Key",
                        "Access-Control-Allow-Methods" : "POST",
                        "Access-Control-Allow-Origin":"*"
                            },
                       "body": json.dumps(return_data),
                    }
                else:
                    return {
                        "statusCode": 400,
                        "body": json.dumps({"error": "Invalid filter keyword"})
                    }
    except Exception as e:
        return {
            "statusCode": 404,
            "headers": {
                "Content-Type": "application/json",
                "Access-Control-Allow-Headers": "Content-Type,X-Api-Key",
                "Access-Control-Allow-Methods": "POST",
                "Access-Control-Allow-Origin": "*"
            },
            "body": f"there was an error, {e}"
        }


# 6-Test Events

In [41]:
## Set up the data
request_event = {

    "body": {
        "data": {
            "action":"get",
            "csv_filename": "schools_list.csv"
        }
    }
}



# Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

Status code is: 200
Result is: "Name,Latitude,Longitude,Radius\nShawlands Primary School,55.82952239999999,-4.2855491,4.0\nPollokshields Primary School,55.84229389999999,-4.2747136,4.0\nHillington Primary School,55.8524158,-4.3582729,4.0\nSandwood Primary School,55.8495065,-4.3621152,4.0\nSir John Maxwell School,55.8255717,-4.2970815,4.0\nCuthbertson Primary School,55.8397751,-4.2651444,4.0\nIbrox Primary School & Nursery Class,55.8517114,-4.3084599,4.0\nAshpark Primary School,55.8077353,-4.3329298,4.0\nTinto Primary School,55.8175865,-4.292825,4.0\nCardonald Primary School,55.8422919,-4.3503377,4.0\nMosspark Primary School,55.8391122,-4.3276337,4.0\nBraidbar Primary School,55.8065805,-4.2837185,4.0\nMerrylee Primary School,55.8166313,-4.274818199999999,4.0\nHoly Cross Primary School,55.836191,-4.254701499999999,4.0\nGiffnock Primary School,55.80277239999999,-4.2937503,4.0\nDarnley Primary School,55.8079233,-4.33972,4.0\nHollybrook Academy,55.8368914,-4.253457,4.0\nGowanbank Primary Sc

In [42]:
request_event = {

     "body":  {
         "data": {
             "action":"store",
             "latitude":99.999999 ,
             "longitude":99.999999 ,
             "radius":99,
             "datalist":"schools_list"
         }
     }
}
   # Run the lambda_handler with a context of None
result = lambda_handler(request_event, None)
print("Status code is:", result["statusCode"])
print("Result is:", result["body"])

List of schools saved to S3 at s3.
Status code is: 200
Result is: {"message": "School locations retrieved and saved to S3"}
